# Import libraries

In [1]:
import subprocess
import os
from helpers import parse_variables
import pandas as pd
import numpy as np

# Load simulation parameters

In [2]:
dict = parse_variables('geno_simulation.txt')
G = int(dict['G'])
L = int(dict['L'])
c = int(dict['c'])
if 'k' not in globals():
    k = int(dict['k'])
    
if 'M' not in globals():
    M = float(dict['M'])
HWE = int(dict['HWE'])

nr_humans = int(dict['nr_humans'])
nr_snps = int(dict['nr_snps'])
bottleneck_nr = int(dict['bottleneck_nr'])

# Thresholds
very_rare_threshold_L = float(dict['very_rare_threshold_L'])
very_rare_threshold_H = float(dict['very_rare_threshold_H'])

rare_threshold_L = float(dict['rare_threshold_L'])
rare_threshold_H = float(dict['rare_threshold_H'])

common_threshold_L = float(dict['common_threshold_L'])
common_threshold_H = float(dict['common_threshold_H'])

number_of_snps = (G*L)/2 # one loci per chromosome
number_of_individuals = c*k*k

very_rare = pd.read_pickle(f"data/G{G}_L{L}_c{c}_k{k}_M{M}_HWE{HWE}/genotype/01_veryrare_genotype_AF_{very_rare_threshold_L}_{very_rare_threshold_H}.pkl")
rare = pd.read_pickle(f"data/G{G}_L{L}_c{c}_k{k}_M{M}_HWE{HWE}/genotype/01_rare_genotype_AF_{rare_threshold_L}_{rare_threshold_H}.pkl")
common = pd.read_pickle(f"data/G{G}_L{L}_c{c}_k{k}_M{M}_HWE{HWE}/genotype/01_common_genotype_AF_{common_threshold_L}_{common_threshold_H}.pkl")

complete = pd.concat([common, rare, very_rare], axis=1)
complete = ((complete*2)-1)

In [3]:
path_bottle = f"data/G{G}_L{L}_c{c}_k{k}_M{M}_HWE{HWE}/phenotype/abyss_bottleneck"
bottle_file = [f for f in os.listdir(path_bottle) if int(f.split("_")[2]) ==  bottleneck_nr][0]
elapsed_time_bottleneck = float(bottle_file.split('_')[3].split('seconds')[0])
bottle = pd.read_pickle(f"{path_bottle}/{bottle_file}")

In [4]:
bottle

,dim1,dim2,dim3,dim4,dim5,dim6,dim7,dim8,dim9,dim10,...,dim56,dim57,dim58,dim59,dim60,dim61,dim62,dim63,dim64,cluster
0,-0.765476,1.249960,0.898525,0.405163,-0.447200,-0.370296,-0.106457,0.030834,1.355991,0.286983,...,0.405096,-0.561182,-0.142547,-0.504342,0.656515,-0.517348,-0.096568,-0.173794,-0.696391,0
1,-0.682315,-0.447598,0.137246,1.024404,1.080366,0.359358,0.994212,0.297233,0.553536,-0.626948,...,-0.551435,0.812257,-0.769628,0.793806,1.282208,-0.502998,0.616734,-0.471180,0.365209,0
2,0.107420,0.842467,1.121887,-0.240508,1.447541,-0.323402,-0.307228,0.042146,0.017074,0.124375,...,-0.109742,0.481677,-0.284843,0.350055,0.684395,-0.455911,1.423192,-0.453608,-0.558621,0
3,0.904169,1.185245,-0.280036,1.006666,1.096568,-0.498515,1.302604,1.111808,0.716737,1.234424,...,-0.288333,0.640048,-0.710708,-0.528326,0.102872,0.517202,0.817534,0.670589,0.803046,0
4,-0.861575,-0.460620,-0.478165,-0.430176,1.124293,1.015549,0.788779,-0.161121,-0.068093,1.668627,...,0.126951,-0.611034,0.237804,0.326888,1.078374,0.258653,-0.106712,0.458515,-0.616256,0
5,-0.683014,0.291857,-0.231474,0.044364,-0.030912,-0.093658,-0.312488,0.295214,-0.270874,-0.171668,...,0.051853,0.643333,-0.855006,0.434796,1.175496,-0.322475,0.074253,0.975136,-0.746030,0
6,-0.111004,0.101331,0.799258,0.609165,0.647986,1.417124,0.759185,0.031739,1.145000,0.979373,...,-0.620951,0.057239,-0.671952,1.784115,0.250748,-0.231300,0.906291,0.470298,-0.321149,0
7,-0.328082,0.313960,0.487320,0.258439,0.706826,0.045608,0.530123,0.729606,0.193852,0.286837,...,-0.289135,0.144059,-0.580570,-0.111746,0.642192,-0.419108,0.448043,0.015484,-0.473052,0
8,-0.092885,0.521105,1.361751,-0.402597,-0.131294,-0.246483,0.681757,1.547320,0.633617,1.263240,...,-0.484449,0.112575,-0.725399,-0.652940,0.758090,-0.016996,0.678954,-0.507790,-0.683449,0
9,0.684866,-0.195693,0.044988,-0.380943,1.305807,0.386538,1.048939,1.240112,0.855221,0.926972,...,-0.468659,0.601291,-0.596043,0.924420,0.071099,-0.710777,0.073598,1.218611,-0.530594,0


In [5]:
path_pops_estimated_lds = f"data/G{G}_L{L}_c{c}_k{k}_M{M}_HWE{HWE}/genotype/LD_blocks_estimated_mafs/"

In [6]:
L*G

10000

In [7]:
q2s_pop = []
twopqs_pop = []
p2s_pop = []

time_q2 = 0.0
time_p2 = 0.0
time_2pq = 0.0

for pop in os.listdir(path_pops_estimated_lds):
    print(pop)
    bottle_index = bottle[bottle['cluster']==int(pop)]
    path_estimated_lds = path_pops_estimated_lds + "/" + pop
    q2_files = [f for f in os.listdir(path_estimated_lds) if f.split(f"_")[6] == 'q2']
    q2_files = sorted(q2_files, key=lambda x: int(x.split('_')[0]))
    p2_files = [f for f in os.listdir(path_estimated_lds) if f.split(f"_")[6] == 'p2']
    p2_files = sorted(p2_files, key=lambda x: int(x.split('_')[0]))
    
    twopq_files = [f for f in os.listdir(path_estimated_lds) if f.split(f"_")[6] == '2pq']
    twopq_files = sorted(twopq_files, key=lambda x: int(x.split('_')[0]))
    
    
    q2s = []
    for q2_file in q2_files:
        time_q2 += float(q2_file.split('_pop_')[1].split("seconds")[0])
        path_q2_file = path_estimated_lds + '/' + q2_file
        q2 = pd.read_pickle(path_q2_file)
        q2s.append(q2)
    
    q2s = pd.concat(q2s, axis=1)
    q2s = q2s[list(complete.columns)]
    q2s_pop.append(q2s)

    p2s = []
    for p2_file in p2_files:
        time_p2 += float(p2_file.split('_pop_')[1].split("seconds")[0])

        path_p2_file = path_estimated_lds + '/' + p2_file
        p2 = pd.read_pickle(path_p2_file)
        p2s.append(p2)

    p2s = pd.concat(p2s, axis=1)
    p2s = p2s[list(complete.columns)]
    p2s_pop.append(p2s)

    
    twopqs = []
    for twopq_file in twopq_files:
        time_2pq += float(twopq_file.split('_pop_')[1].split("seconds")[0])
        path_2pq_file = path_estimated_lds + '/' + twopq_file
        twopq = pd.read_pickle(path_2pq_file)
        twopqs.append(twopq)

    twopqs = pd.concat(twopqs, axis=1)
    twopqs = twopqs[list(complete.columns)]
    twopqs_pop.append(twopqs)


q2s = pd.concat(q2s_pop, axis=0)
q2s = q2s.sort_index()

p2s = pd.concat(p2s_pop, axis=0)
p2s = p2s.sort_index()

twopqs = pd.concat(twopqs_pop, axis=0)
twopqs = twopqs.sort_index()

0


In [8]:
path_output = f"data/G{G}_L{L}_c{c}_k{k}_M{M}_HWE{HWE}/genotype/"

In [9]:
os.system(f"rm -rf {path_output}/estimated*")

0

In [10]:
q2s.to_pickle(f"{path_output}/estimated_q2s_via_esti_pop_{time_q2}seconds.pkl")
p2s.to_pickle(f"{path_output}/estimated_p2s_via_esti_pop_{time_p2}seconds.pkl")
twopqs.to_pickle(f"{path_output}/estimated_2pqs_via_esti_pop_{time_2pq}seconds.pkl")